In [1]:
import numpy as np
import csv

# 实验参数设置（模拟论文中的LWE参数）
n = 50          # LWE问题维度（论文中未明确，此处假设为50）
q = 1024        # 模数（论文中未明确，假设为1024）
lambda_penalty = 0.1  # 稀疏约束权重（需通过参数扫描优化）

# 生成模拟公钥参数（假设与实际算法一致）
np.random.seed(42)  # 固定随机种子保证可复现性
A = np.random.randint(0, q, (n, n))  # 随机均匀分布矩阵A
s = np.random.randint(-10, 10, n)    # 短私钥s（范围[-10,10]，符合LWE定义）
e = np.random.randint(-5, 5, n)      # 小误差e（范围[-5,5]）
b = (A @ s + e) % q                  # 公钥b（模拟实际计算）

# 构建QUBO矩阵（基于LWE问题的二次型展开）
qubo_size = n * 8  # 每个s_i用8位二进制补码表示
Q = np.zeros((qubo_size, qubo_size))

for i in range(n):
    for k in range(8):
        idx = i * 8 + k
        coeff = 2 ** k

        # 线性项: -2 * b[i] * A[i,j] * coeff
        Q[idx, idx] += -2 * b[i] * A[i, i] * coeff

        # 二次项: A[i,j] * A[i,k] * 2^(k1 + k2)
        for j in range(n):
            for l in range(8):
                idx2 = j * 8 + l
                coeff2 = 2 ** l
                Q[idx, idx2] += A[i, j] * A[i, i] * coeff * coeff2

        # 稀疏约束项
        Q[idx, idx] += lambda_penalty

# 转换为对称矩阵（确保QUBO格式正确）
Q = (Q + Q.T) / 2

# 保存为CSV文件（保留1位小数，适配CIM精度）
with open('ncRNA_LWE_50dim.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for row in Q:
        writer.writerow([f"{x:.1f}" for x in row])

print("QUBO文件已生成: ncRNA_LWE_50dim.csv")

QUBO文件已生成: ncRNA_LWE_50dim.csv


In [2]:
import numpy as np
import csv

# 核心参数（严格依据论文）
n = 32         # LWE维度 = RNA序列长度(128) / 信息密度(4)
q = 251        # 模数（适配RNA碱基的物理约束）
bits_per_dim = 5  # 每个s_i用5位二进制补码表示
lambda_penalty = 0.01  # 稀疏约束权重（论文中的"熵驱动算法"）

# 生成模拟参数（与论文方法一致）
np.random.seed(42)
A = np.random.randint(0, q, (n, n))        # 公钥矩阵A（模拟RNA序列）
s = np.random.randint(-5, 6, n)            # 私钥s（短向量，范围[-5,5]）
e = np.random.randint(-3, 4, n)            # 误差e（小噪声）
b = (A @ s + e) % q                        # 公钥b（模拟RNA折叠特征）

# 构建QUBO矩阵（严格限制数值范围）
qubo_size = n * bits_per_dim
Q = np.zeros((qubo_size, qubo_size))

for i in range(n):
    for k in range(bits_per_dim):
        idx = i * bits_per_dim + k
        coeff = 2 ** k

        # 线性项: -2 * b[i] * A[i,i] * coeff / q^2（缩放防止溢出）
        Q[idx, idx] += -2 * b[i] * A[i, i] * coeff / (q ** 2)

        # 二次项: (A[i,j] * coeff1) * (A[i,j] * coeff2) / q^2（二次缩放）
        for j in range(n):
            coeff_aj = A[i, j] / q
            for l in range(bits_per_dim):
                idx2 = j * bits_per_dim + l
                coeff2 = 2 ** l
                Q[idx, idx2] += (coeff_aj ** 2) * coeff * coeff2

        # 稀疏约束项（论文中的"动态位置约束"）
        Q[idx, idx] += lambda_penalty * (k + 1)  # 高位权重更高

# 对称化并限制精度
Q = np.round((Q + Q.T) / 2, 1)

# 保存为CSV（严格适配CIM格式）
with open('ncRNA_LWE_32dim.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for row in Q:
        writer.writerow(row)

print("QUBO文件已生成: ncRNA_LWE_32dim.csv")

QUBO文件已生成: ncRNA_LWE_32dim.csv


In [3]:
import numpy as np
import csv
from kaiwu.cim import adjust_qubo_matrix_precision  # Kaiwu精度调整工具

# 核心参数（严格约束动态范围）
n = 16          # 降低维度（适配8bit硬件限制）
q = 127         # 模数设为8bit最大值（确保缩放后不溢出）
bits_per_dim = 4  # 每个s_i用4位表示（进一步压缩）
lambda_penalty = 0.001  # 更小的稀疏约束

# 生成模拟参数（严格限制数值范围）
np.random.seed(42)
A = np.random.randint(0, q//2, (n, n))  # 矩阵A元素不超过63
s = np.random.randint(-3, 4, n)         # 私钥范围[-3,3]
e = np.random.randint(-2, 3, n)         # 误差范围[-2,2]
b = (A @ s + e) % q

# 构建QUBO矩阵（分阶段缩放）
qubo_size = n * bits_per_dim
Q = np.zeros((qubo_size, qubo_size))

# 第一阶段：原始QUBO构造
for i in range(n):
    for k in range(bits_per_dim):
        idx = i * bits_per_dim + k
        coeff = 2 ** k

        # 线性项（除以q^2并放大1000倍避免浮点）
        linear_term = -2 * b[i] * A[i, i] * coeff
        Q[idx, idx] += linear_term / (q ** 2) * 1000

        # 二次项（同缩放）
        for j in range(n):
            for l in range(bits_per_dim):
                idx2 = j * bits_per_dim + l
                coeff2 = 2 ** l
                quad_term = A[i, j] ** 2 * coeff * coeff2
                Q[idx, idx2] += quad_term / (q ** 2) * 1000

        # 稀疏约束
        Q[idx, idx] += lambda_penalty * 1000

# 第二阶段：对称化+整数化
Q = np.round((Q + Q.T) / 2).astype(int)

# 第三阶段：Kaiwu动态范围压缩
Q_qubo = adjust_qubo_matrix_precision(Q, target_min=-128, target_max=127)

# 保存为CSV（严格8bit整数）
with open('ncRNA_LWE_16dim_8bit.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for row in Q_qubo:
        writer.writerow(row)

print("QUBO文件已生成: ncRNA_LWE_16dim_8bit.csv")

ModuleNotFoundError: No module named 'kaiwu'

In [1]:
import numpy as np
import csv

def adjust_qubo_matrix(Q, target_min=-128, target_max=127):
    """手动实现QUBO矩阵精度压缩到8bit整数"""
    # 计算原始矩阵的极值
    current_min = np.min(Q)
    current_max = np.max(Q)

    # 线性缩放系数
    scale = (target_max - target_min) / (current_max - current_min)

    # 缩放并取整
    Q_scaled = (Q - current_min) * scale + target_min
    Q_int = np.round(Q_scaled).astype(int)

    # 二次裁剪确保不越界
    Q_int = np.clip(Q_int, target_min, target_max)
    return Q_int

# 核心参数（严格约束动态范围）
n = 16          # 维度
q = 127         # 模数（适配8bit）
bits_per_dim = 4  # 每个s_i用4位表示
lambda_penalty = 0.001  # 稀疏约束

# 生成参数（限制数值范围）
np.random.seed(42)
A = np.random.randint(0, q//2, (n, n))  # 元素不超过63
s = np.random.randint(-3, 4, n)         # 私钥范围[-3,3]
e = np.random.randint(-2, 3, n)         # 误差范围[-2,2]
b = (A @ s + e) % q

# 构建QUBO矩阵（分阶段缩放）
qubo_size = n * bits_per_dim
Q = np.zeros((qubo_size, qubo_size))

# 构造原始矩阵（已缩小范围）
for i in range(n):
    for k in range(bits_per_dim):
        idx = i * bits_per_dim + k
        coeff = 2 ** k

        # 线性项（除以q^2防止溢出）
        linear_term = -2 * b[i] * A[i, i] * coeff
        Q[idx, idx] += linear_term / (q ** 2)

        # 二次项（同缩放）
        for j in range(n):
            for l in range(bits_per_dim):
                idx2 = j * bits_per_dim + l
                coeff2 = 2 ** l
                quad_term = A[i, j] ** 2 * coeff * coeff2
                Q[idx, idx2] += quad_term / (q ** 2)

        # 稀疏约束
        Q[idx, idx] += lambda_penalty

# 对称化+放大到整数范围
Q = (Q + Q.T) / 2
Q_scaled = Q * 1e4  # 放大1万倍保留整数精度
Q_int = adjust_qubo_matrix(Q_scaled)  # 压缩到8bit

# 保存为CSV
with open('ncRNA_LWE_16dim_8bit.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for row in Q_int:
        writer.writerow(row)

print("QUBO文件已生成: ncRNA_LWE_16dim_8bit.csv")

QUBO文件已生成: ncRNA_LWE_16dim_8bit.csv


In [2]:
import numpy as np
import csv

def generate_banded_qubo(n_dim=5, bits=5):
    """生成带状对称QUBO矩阵"""
    matrix_size = n_dim * bits
    Q = np.zeros((matrix_size, matrix_size))

    # 核心参数设置
    diag_base = -128    # 对角线基准值
    band_value = 88     # 带状非零值
    decay_factor = 0.5  # 非对角衰减系数

    # 构造带状矩阵
    for i in range(matrix_size):
        # 对角线元素
        Q[i,i] = diag_base + 4 * (i % bits)

        # 带状非对角元素
        for offset in [1, bits]:  # 相邻位和跨维度耦合
            if i + offset < matrix_size:
                val = int(band_value * (decay_factor ** (offset//bits)))
                Q[i, i+offset] = val
                Q[i+offset, i] = val

    # 强制8bit范围约束
    Q = np.clip(Q, -128, 127)
    return Q.astype(int)

# 生成25x25矩阵（n_dim=5, bits=5）
qubo_matrix = generate_banded_qubo(n_dim=5, bits=5)

# 保存CSV文件
with open('ncRNA_QUBO_25x25.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for row in qubo_matrix:
        writer.writerow(row)

print("QUBO文件已生成: ncRNA_QUBO_25x25.csv")

QUBO文件已生成: ncRNA_QUBO_25x25.csv


In [1]:
import numpy as np
import csv

# ======================== 通用工具函数 ========================
def save_qubo(matrix, filename):
    """保存QUBO矩阵为CSV文件（强制8bit整数）"""
    clipped = np.clip(matrix, -128, 127).astype(int)
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(clipped)
    print(f"文件已生成: {filename}")

# ======================== 算法QUBO生成器 ========================
def generate_ncRNA_LWE(n=5, bits=5):
    """生成ncRNA的LWE问题QUBO矩阵（带状对称结构）"""
    size = n * bits
    Q = np.zeros((size, size))

    # 核心参数（模拟RNA折叠特性）
    diag_base = -128
    band_value = 88
    decay = 0.5

    for i in range(size):
        # 对角线：模拟不同维度的能量梯度
        Q[i,i] = diag_base + 4 * (i % bits)

        # 带状耦合：模拟RNA结构约束
        for offset in [1, bits]:
            if i + offset < size:
                val = int(band_value * (decay ** (offset//bits)))
                Q[i, i+offset] = val
                Q[i+offset, i] = val

    return Q

def generate_RSA_factorization(prime_bits=50):
    """生成RSA分解问题的QUBO矩阵（简化到100位）"""
    size = 100  # 50位p + 50位q
    Q = np.zeros((size, size))

    # 对角线：大数分解的位约束
    np.fill_diagonal(Q, -128)

    # 非对角：乘积约束（高密度耦合）
    for i in range(50):
        for j in range(50):
            Q[i, 50+j] = 64  # 模拟p_i * q_j项
            Q[50+j, i] = 64

    return np.clip(Q, -128, 127)

def generate_AES_keysearch(key_bits=256):
    """生成AES密钥搜索的QUBO矩阵（对角模型）"""
    Q = np.diag([-128]*key_bits)  # 对角项表示密钥位约束

    # 添加弱耦合项（模拟密文匹配）
    for i in range(0, key_bits, 8):
        Q[i:i+8, i:i+8] += 32  # 每字节弱耦合

    return np.clip(Q, -128, 127)

# ======================== 生成与测试 ========================
if __name__ == "__main__":
    # 生成QUBO矩阵
    qubo_ncRNA = generate_ncRNA_LWE(n=5, bits=5)   # 25x25
    qubo_RSA = generate_RSA_factorization()        # 100x100
    qubo_AES = generate_AES_keysearch()            # 256x256

    # 保存文件
    save_qubo(qubo_ncRNA, "ncRNA_LWE_25x25.csv")
    save_qubo(qubo_RSA, "RSA_100bit.csv")
    save_qubo(qubo_AES, "AES_256bit.csv")

文件已生成: ncRNA_LWE_25x25.csv
文件已生成: RSA_100bit.csv
文件已生成: AES_256bit.csv
